In [1]:
from hashlib import sha1
from Crypto.Cipher import AES
import hmac
from binascii import a2b_hex

def point2str(R, l):
    (rx, ry) = R.xy()
    rxstr = I2OSP(rx, l)
    rystr = I2OSP(ry, l)
    rstr = rxstr + rystr
    return rstr

def padding(s):
    return s + (b"\0" * (AES.block_size - len(s) % AES.block_size)).decode("utf-8")

def keyDerivation(x, l, certi):
    assert l >= 0
    k = l / float(20)
    k = int(ceil(k))
    tmp = ''
    for i in range(0, k):
        tmp += sha1(x + I2OSP(i, 4) + certi).hexdigest()
    return tmp[:l * 2]


def I2OSP(li, l):
    h_str = '%X' % li
    return a2b_hex(h_str.zfill(2 * l))

In [2]:
def keygen():
    d = randint(1, n - 1)
    Q = d * P
    return (Q, d)

In [3]:
def encrypt(Q, m):
    k = randint(1, n - 1)
    R = k * P
    Z = h * k * Q
    l = ceil(math.log(n, 2)/8)
    (zx, zy) = Z.xy()
    zxstr = I2OSP(zx, l)
    rstr = point2str(R, l)
    s = keyDerivation(zxstr, l, rstr)
    k1 = s[0:l]
    k2 = s[l:2*l]
    aes = AES.new(k1)
    C = aes.encrypt(padding(m))
    k_hmac = hmac.new(k2.encode("utf-8"))
    k_hmac.update(C)
    t = k_hmac.digest()
    return [R, C, t]

In [4]:
def decrypt(R, C, t, d):
    Z = h * d * R
    l = ceil(math.log(n, 2)/8)
    (zx, zy) = Z.xy()
    zxstr = I2OSP(zx, l)
    rstr = point2str(R, l)
    s = keyDerivation(zxstr, l, rstr)
    k1 = s[0:l]
    k2 = s[l:2*l]
    k_hmac = hmac.new(k2.encode("utf-8"))
    k_hmac.update(C)
    t1 = k_hmac.digest()
    if t1 != t:
        return False
    aes = AES.new(k1)
    m = aes.decrypt(C)
    return m

In [5]:
F = FiniteField(2**192 - 2**64 - 1)
a  = -3
b  = 0x64210519E59C80E70FA7E9AB72243049FEB8DEECC146B9B1
E  = EllipticCurve(F, [a, b])
P  = E((0x188DA80EB03090F67CBF20EB43A18800F4FF0AFD82FF1012,0x07192B95FFC8DA78631011ED6B24CDD573F977A11E794811))
n  = 0xFFFFFFFFFFFFFFFFFFFFFFFF99DEF836146BC9B1B4D22831
h  = 1
Fn = FiniteField(n)

In [6]:
(Q, d) = keygen()

In [7]:
msg = "Elliptic Curve Cryptography"

In [8]:
[R, C, t] = encrypt(Q, msg)

/home/sage/sage/local/lib/python3.7/hmac.py:153: DeprecationWarning: HMAC() without an explicit digestmod argument is deprecated since Python 3.4, and will be removed in 3.8
  return HMAC(key, msg, digestmod)


In [9]:
decrypt(R, C, t, d)

/home/sage/sage/local/lib/python3.7/hmac.py:153: DeprecationWarning: HMAC() without an explicit digestmod argument is deprecated since Python 3.4, and will be removed in 3.8
  return HMAC(key, msg, digestmod)


b'Elliptic Curve Cryptography\x00\x00\x00\x00\x00'